In [ ]:
import pandas as pd

dt_tag_cleaned=pd.read_csv('dt_tag_cleaned.csv')

dt_tag_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462004 entries, 0 to 462003
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   userId         462004 non-null  int64 
 1   movieId        462004 non-null  int64 
 2   tag            462004 non-null  object
 3   tag_timestamp  462004 non-null  object
dtypes: int64(2), object(2)
memory usage: 14.1+ MB


In [ ]:
dt_tag_cleaned.drop_duplicates(subset=['userId', 'movieId'], keep='last', inplace=True)
dt_tag_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173539 entries, 5 to 462003
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   userId         173539 non-null  int64 
 1   movieId        173539 non-null  int64 
 2   tag            173539 non-null  object
 3   tag_timestamp  173539 non-null  object
dtypes: int64(2), object(2)
memory usage: 6.6+ MB


In [ ]:
usersWithMovies= dt_tag_cleaned[['userId','movieId']]
usersWithMovies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173539 entries, 5 to 462003
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   userId   173539 non-null  int64
 1   movieId  173539 non-null  int64
dtypes: int64(2)
memory usage: 4.0 MB


In [ ]:
print('The total number of distinct movies watched by at least user',len(set(usersWithMovies['movieId'])))
print('The total number of distinct users',len(set(usersWithMovies['userId'])))

The total number of distinct movies watched by at least user 19223
The total number of distinct users 7769


In [ ]:
userMovies=usersWithMovies.groupby('userId')['movieId'].apply(list).reset_index(name="movieList")
userMovies

,userId,movieList
0,18,[4141]
1,65,"[2193, 2662, 3052, 1391, 8529, 353, 521, 592, ..."
2,96,[106696]
3,121,"[4718, 5283, 35836, 1288, 54503, 2706, 46970, ..."
4,129,"[4732, 2706, 80846, 80350, 4247, 8528, 52952, ..."
...,...,...
7764,138414,"[51255, 61240, 8917, 71535, 49772, 44665, 7913..."
7765,138436,"[6283, 67665, 787, 8191, 25943, 26111, 25911, ..."
7766,138437,[77154]
7767,138446,"[837, 3086, 918, 2396, 7045, 55999, 7164, 3489..."


In [ ]:
userMovies['numMoviesPerUser']  = userMovies['movieList'].str.len()
userMovies


,userId,movieList,numMoviesPerUser
0,18,[4141],1
1,65,"[2193, 2662, 3052, 1391, 8529, 353, 521, 592, ...",30
2,96,[106696],1
3,121,"[4718, 5283, 35836, 1288, 54503, 2706, 46970, ...",16
4,129,"[4732, 2706, 80846, 80350, 4247, 8528, 52952, ...",24
...,...,...,...
7764,138414,"[51255, 61240, 8917, 71535, 49772, 44665, 7913...",19
7765,138436,"[6283, 67665, 787, 8191, 25943, 26111, 25911, ...",205
7766,138437,[77154],1
7767,138446,"[837, 3086, 918, 2396, 7045, 55999, 7164, 3489...",9


In [ ]:
print('On average the number of movies watched by a user is', userMovies['numMoviesPerUser'].mean())

On average the number of movies watched by a user is 22.3373664564294


In [ ]:
userMovies[userMovies['numMoviesPerUser']>1000]

,userId,movieList,numMoviesPerUser
94,1741,"[4970, 7458, 1221, 2019, 1232, 95837, 2012, 10...",2410
2591,45928,"[4484, 26181, 6983, 1131, 4979, 1003, 6721, 26...",1510
3283,58612,"[161, 6659, 27879, 1220, 1076, 1200, 41627, 26...",1366
3920,70201,"[119589, 88352, 114567, 112495, 4881, 2437, 10...",10515
4984,88738,"[71158, 72612, 48082, 731, 94220, 2910, 7354, ...",7365
5423,96370,"[7835, 2121, 1545, 4232, 1041, 1263, 3994, 122...",1234
6806,120937,"[8784, 4085, 71732, 45431, 288, 45335, 64197, ...",1140
6941,123297,"[5679, 31991, 112467, 77223, 6460, 79923, 6895...",5630
7044,124998,"[25999, 2211, 7728, 6908, 25777, 915, 5234, 10...",1370
7147,127138,"[1120, 448, 52938, 1069, 803, 52813, 2176, 206...",1006


For our movie baskets, we will exclude those with two or fewer movies, as they may not provide enough data to identify frequent movie. Additionally, baskets with fewer than 10K movies will be omitted, as they could result in an overwhelming number of potential candidate movies in the Apriori algorithm.

In [ ]:
users_with_plentyNumMovies = userMovies[(userMovies['numMoviesPerUser'] > 2)& (userMovies['numMoviesPerUser'] < 10000) ]
users_with_plentyNumMovies

,userId,movieList,numMoviesPerUser
1,65,"[2193, 2662, 3052, 1391, 8529, 353, 521, 592, ...",30
3,121,"[4718, 5283, 35836, 1288, 54503, 2706, 46970, ...",16
4,129,"[4732, 2706, 80846, 80350, 4247, 8528, 52952, ...",24
6,190,"[1270, 2012, 2011]",3
8,208,"[41573, 8645, 40819, 27727, 72]",5
...,...,...,...
7760,138307,"[5673, 4973, 1916, 7361]",4
7761,138345,"[904, 58559, 1104, 3617, 2485, 527, 750, 1125]",8
7764,138414,"[51255, 61240, 8917, 71535, 49772, 44665, 7913...",19
7765,138436,"[6283, 67665, 787, 8191, 25943, 26111, 25911, ...",205


In [ ]:
#movie_baskets=userMovies['movieList'].tolist()
movie_baskets=users_with_plentyNumMovies['movieList'].tolist()
print('The number of baskets is:',len(movie_baskets))
distinct_movies={ele for movie_basket in movie_baskets for ele in movie_basket}



The number of baskets is: 3767


In [ ]:
import itertools
import pandas as pd

def getNumBasketsForItem(item, baskets):
    """
    Calculate the support of a single item in the baskets.
    """
    return sum(item in basket for basket in baskets) / len(baskets)

def getNumBasketsforItems(item_set, baskets):
    """
    Calculate the support of an itemset (made up of more than 1 item) in the baskets.
    """
    item_set = set(item_set)  # Convert to set once for efficiency
    return sum(item_set.issubset(basket) for basket in baskets) / len(baskets)

def aprioriFromScratch(baskets_encoded, support_threshold):
    """
    Implements the Apriori algorithm to find frequent itemsets.

    Parameters:
    - baskets_encoded: List of transactions (baskets) with encoded items.
    - support_threshold: Minimum support threshold for itemsets.

    Returns:
    - A DataFrame containing frequent itemsets and their support values.
    """
    # Initialize distinct items and calculate support for single items
    distinct_items = list({item for basket in baskets_encoded for item in basket})
    item_supports = [getNumBasketsForItem(item, baskets_encoded) for item in distinct_items]

    # Initialize result DataFrame and candidate/frequent itemsets
    result = pd.DataFrame(columns=['ItemSet', 'Support'])
    k = 1

    # Create DataFrame for single-item candidates
    candidates = pd.DataFrame({'ItemSet': [{item} for item in distinct_items], 'Support': item_supports})
    frequent_itemsets = candidates[candidates['Support'] > support_threshold].reset_index(drop=True)

    # Keep track of frequent itemsets for each iteration
    dataframes = {k: frequent_itemsets}

    # Iteratively find frequent itemsets of size k
    while not frequent_itemsets.empty:
        k += 1

        # Generate combinations of itemsets of size k
        previous_itemsets = frequent_itemsets['ItemSet'].tolist()
        candidate_itemsets = [
            set(a).union(b)  # Merge sets to generate new candidates
            for a, b in itertools.combinations(previous_itemsets, 2)
            if len(set(a).union(b)) == k
        ]

        # Deduplicate candidate itemsets
        candidate_itemsets = list({frozenset(c) for c in candidate_itemsets})

        # Calculate support for candidate itemsets
        itemset_supports = [getNumBasketsforItems(itemset, baskets_encoded) for itemset in candidate_itemsets]
        candidates = pd.DataFrame({'ItemSet': candidate_itemsets, 'Support': itemset_supports})

        # Filter itemsets with support above the threshold
        frequent_itemsets = candidates[candidates['Support'] > support_threshold]
        if not frequent_itemsets.empty:
            dataframes[k] = frequent_itemsets

    # Concatenate all frequent itemsets into the final result
    for df in dataframes.values():
        result = pd.concat([result, df], ignore_index=True)

    # Convert `frozenset` back to `set` in the final DataFrame
    result['ItemSet'] = result['ItemSet'].apply(set)
    print('The number of frequents items is ', result.shape[0])

    return result.reset_index(drop=True)


In [ ]:
import pandas as pd


# Run Apriori algorithm implemented by Matteo
resultApriori = aprioriFromScratch(movie_baskets, support_threshold=0.03)

resultApriori


The number of frequents items is  169


<ipython-input-11-a88d1ea5498f>:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, df], ignore_index=True)


,ItemSet,Support
0,{1},0.038758
1,{32},0.070879
2,{47},0.074330
3,{50},0.078312
4,{110},0.043536
...,...,...
164,"{7361, 2959}",0.030263
165,"{296, 318}",0.044598
166,"{296, 32587}",0.030528
167,"{32, 50}",0.030263


We check if the itemsets and their support values found by my implementation of Apriori are the same as those found using the Apriori from the mlxtend library.
Taking into account that the Apriori from the mxlxtend library use itemsset as  frozenset , in the result of the Apriori implemented by hand a new column of the itemset is added and converted into frozenset.
In such way, we can compare whether the results in terms of possible frequent items found and their relative support are the same

In [ ]:
resultApriori['itemsets']=resultApriori['ItemSet']
resultApriori['itemsets']=resultApriori['itemsets'].apply(lambda x: frozenset(x))
resultApriori

,ItemSet,Support,itemsets
0,{1},0.038758,(1)
1,{32},0.070879,(32)
2,{47},0.074330,(47)
3,{50},0.078312,(50)
4,{110},0.043536,(110)
...,...,...,...
164,"{7361, 2959}",0.030263,"(7361, 2959)"
165,"{296, 318}",0.044598,"(296, 318)"
166,"{296, 32587}",0.030528,"(296, 32587)"
167,"{32, 50}",0.030263,"(32, 50)"


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

te = TransactionEncoder()
te_ary = te.fit(movie_baskets).transform(movie_baskets)
df_one_hot = pd.DataFrame(te_ary, columns=te.columns_)
df_one_hot
frq_items = apriori(df_one_hot, min_support = 0.03, use_colnames = True)
frq_items

,support,itemsets
0,0.038758,(1)
1,0.070879,(32)
2,0.074330,(47)
3,0.078312,(50)
4,0.043536,(110)
...,...,...
164,0.030263,"(7361, 2959)"
165,0.030263,"(4226, 4878)"
166,0.034776,"(5952, 4993)"
167,0.032387,"(4993, 7153)"


In [ ]:
a=set(resultApriori['Support'].values)
b=set(frq_items['support'].values)
a==b


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


True

In [ ]:
a=set(resultApriori['itemsets'].values)
b=set(frq_items['itemsets'].values)
a==b


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


True

After running the Apriori algorithm we find the rules associated to recommend movies given a certain confidence

In [ ]:


from itertools import combinations

def generate_association_rules(resultApriori, min_confidence):
    """
    Generate association rules from a DataFrame of frequent itemsets.

    Parameters:
        frq_items (pd.DataFrame): DataFrame containing frequent itemsets and their support.
        min_confidence (float): Minimum confidence threshold for rules.

    Returns:
        List of rules with their metrics (support, confidence, lift).
    """
    rules = []

    # Map itemsets to their support for quick lookup
    support_lookup = {tuple(sorted(row['ItemSet'])): row['Support'] for _, row in resultApriori.iterrows()}

    # Iterate through each frequent itemset
    for _, row in resultApriori.iterrows():
        itemset = tuple(sorted(row['ItemSet']))
        itemset_support = row['Support']

        # Generate all possible non-empty subsets of the itemset
        subsets = [set(x) for i in range(1, len(itemset)) for x in combinations(itemset, i)]

        for subset in subsets:
            # A = subset, B = itemset - subset
            A = tuple(sorted(subset))
            B = tuple(sorted(set(itemset) - subset))

            # Calculate metrics
            A_support = support_lookup.get(A, 0)
            B_support = support_lookup.get(B, 0)

            if A_support > 0:  # Avoid division by zero
                confidence = itemset_support / A_support
                lift = confidence / B_support if B_support > 0 else 0

                # Add rule if it meets the confidence threshold
                if confidence >= min_confidence:
                    rules.append({
                        'antecedent': A,
                        'consequent': B,
                        'support': itemset_support,
                        'confidence': confidence,
                        'lift': lift
                    })
    print('The number of rules is' ,len(rules))

    for rule in rules:
      print(f"Rule: {rule['antecedent']} => {rule['consequent']}, "
          f"Support: {rule['support']:.2f}, Confidence: {rule['confidence']:.2f}, Lift: {rule['lift']:.2f}")

    return pd.DataFrame(rules)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print('By running Apriori  with support_threshold=0.03 and association rules with  min_confidence=0.5 we get')

# Generate rules
rules = generate_association_rules(resultApriori, min_confidence=0.5)


rules

By running Apriori  with support_threshold=0.03 and association rules with  min_confidence=0.5 we get
The number of rules is 8
Rule: (4993,) => (5952,), Support: 0.03, Confidence: 0.62, Lift: 12.65
Rule: (5952,) => (4993,), Support: 0.03, Confidence: 0.71, Lift: 12.65
Rule: (4993,) => (7153,), Support: 0.03, Confidence: 0.58, Lift: 9.99
Rule: (7153,) => (4993,), Support: 0.03, Confidence: 0.56, Lift: 9.99
Rule: (858,) => (318,), Support: 0.03, Confidence: 0.50, Lift: 5.10
Rule: (5952,) => (7153,), Support: 0.03, Confidence: 0.67, Lift: 11.60
Rule: (7153,) => (5952,), Support: 0.03, Confidence: 0.57, Lift: 11.60
Rule: (1089,) => (296,), Support: 0.04, Confidence: 0.54, Lift: 4.24


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedent,consequent,support,confidence,lift
0,"(4993,)","(5952,)",0.034776,0.617925,12.650661
1,"(5952,)","(4993,)",0.034776,0.711957,12.650661
2,"(4993,)","(7153,)",0.032387,0.575472,9.989870
3,"(7153,)","(4993,)",0.032387,0.562212,9.989870
4,"(858,)","(318,)",0.030263,0.502203,5.099184
5,"(5952,)","(7153,)",0.032652,0.668478,11.604413
6,"(7153,)","(5952,)",0.032652,0.566820,11.604413
7,"(1089,)","(296,)",0.036368,0.535156,4.244071


We are attempting to generate new rules by modifying the minimum support in the Apriori algorithm as well as adjusting the confidence threshold.




In [ ]:
resultApriori = aprioriFromScratch(movie_baskets, support_threshold=0.025)
resultApriori

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The number of frequents items is  280


<ipython-input-11-a88d1ea5498f>:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, df], ignore_index=True)


,ItemSet,Support
0,{1},0.038758
1,{29},0.026015
2,{32},0.070879
3,{47},0.074330
4,{50},0.078312
...,...,...
275,"{4226, 47}",0.028405
276,"{32, 50}",0.030263
277,"{296, 356}",0.033979
278,"{32, 2571}",0.027077


In [ ]:
print('By running Apriori  with support_threshold=0.025 and association rules with  min_confidence=0.5 we get')
rules = generate_association_rules(resultApriori, min_confidence=0.5)
rules

By running Apriori  with support_threshold=0.025 and association rules with  min_confidence=0.5 we get
The number of rules is 21
Rule: (4993,) => (5952,), Support: 0.03, Confidence: 0.62, Lift: 12.65
Rule: (5952,) => (4993,), Support: 0.03, Confidence: 0.71, Lift: 12.65
Rule: (260,) => (1196,), Support: 0.03, Confidence: 0.61, Lift: 13.68
Rule: (1196,) => (260,), Support: 0.03, Confidence: 0.66, Lift: 13.68
Rule: (1214,) => (2959,), Support: 0.03, Confidence: 0.51, Lift: 4.53
Rule: (4993,) => (7153,), Support: 0.03, Confidence: 0.58, Lift: 9.99
Rule: (7153,) => (4993,), Support: 0.03, Confidence: 0.56, Lift: 9.99
Rule: (589,) => (1240,), Support: 0.03, Confidence: 0.51, Lift: 11.93
Rule: (1240,) => (589,), Support: 0.03, Confidence: 0.61, Lift: 11.93
Rule: (858,) => (318,), Support: 0.03, Confidence: 0.50, Lift: 5.10
Rule: (5952,) => (7153,), Support: 0.03, Confidence: 0.67, Lift: 11.60
Rule: (7153,) => (5952,), Support: 0.03, Confidence: 0.57, Lift: 11.60
Rule: (1089,) => (296,), Supp

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedent,consequent,support,confidence,lift
0,"(4993,)","(5952,)",0.034776,0.617925,12.650661
1,"(5952,)","(4993,)",0.034776,0.711957,12.650661
2,"(260,)","(1196,)",0.029466,0.606557,13.682046
3,"(1196,)","(260,)",0.029466,0.664671,13.682046
4,"(1214,)","(2959,)",0.025484,0.505263,4.531729
5,"(4993,)","(7153,)",0.032387,0.575472,9.989870
6,"(7153,)","(4993,)",0.032387,0.562212,9.989870
7,"(589,)","(1240,)",0.026281,0.512953,11.927749
8,"(1240,)","(589,)",0.026281,0.611111,11.927749
9,"(858,)","(318,)",0.030263,0.502203,5.099184


In [ ]:
print('By running Apriori  with support_threshold=0.025 and association rules with  min_confidence=0.7 we get')
rules = generate_association_rules(resultApriori, min_confidence=0.7)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


By running Apriori  with support_threshold=0.025 and association rules with  min_confidence=0.7 we get
The number of rules is 4
Rule: (5952,) => (4993,), Support: 0.03, Confidence: 0.71, Lift: 12.65
Rule: (4993, 5952) => (7153,), Support: 0.03, Confidence: 0.83, Lift: 14.44
Rule: (4993, 7153) => (5952,), Support: 0.03, Confidence: 0.89, Lift: 18.29
Rule: (5952, 7153) => (4993,), Support: 0.03, Confidence: 0.89, Lift: 15.75


,antecedent,consequent,support,confidence,lift
0,"(5952,)","(4993,)",0.034776,0.711957,12.650661
1,"(4993, 5952)","(7153,)",0.028935,0.832061,14.444120
2,"(4993, 7153)","(5952,)",0.028935,0.893443,18.291295
3,"(5952, 7153)","(4993,)",0.028935,0.886179,15.746395


In [ ]:
resultApriori = aprioriFromScratch(movie_baskets, support_threshold=0.018)
resultApriori

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The number of frequents items is  771


<ipython-input-11-a88d1ea5498f>:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, df], ignore_index=True)


,ItemSet,Support
0,{1},0.038758
1,{6},0.018317
2,{29},0.026015
3,{32},0.070879
4,{39},0.020441
...,...,...
766,"{296, 2858, 318}",0.020972
767,"{296, 593, 50}",0.020175
768,"{32, 593, 47}",0.018582
769,"{296, 50, 4226}",0.019644


In [ ]:
print('By running Apriori  with support_threshold=0.018 and association rules with  min_confidence=0.7 we get')
rules = generate_association_rules(resultApriori, min_confidence=0.7)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


By running Apriori  with support_threshold=0.018 and association rules with  min_confidence=0.7 we get
The number of rules is 32
Rule: (2115,) => (1291,), Support: 0.02, Confidence: 0.73, Lift: 19.81
Rule: (5952,) => (4993,), Support: 0.03, Confidence: 0.71, Lift: 12.65
Rule: (2571, 5952) => (7153,), Support: 0.02, Confidence: 0.76, Lift: 13.26
Rule: (2571, 7153) => (5952,), Support: 0.02, Confidence: 0.85, Lift: 17.40
Rule: (318, 32587) => (296,), Support: 0.02, Confidence: 0.77, Lift: 6.12
Rule: (2571, 7153) => (4993,), Support: 0.02, Confidence: 0.85, Lift: 15.10
Rule: (593, 858) => (296,), Support: 0.02, Confidence: 0.74, Lift: 5.88
Rule: (593, 858) => (318,), Support: 0.02, Confidence: 0.77, Lift: 7.86
Rule: (260, 1196) => (1210,), Support: 0.02, Confidence: 0.75, Lift: 19.29
Rule: (260, 1210) => (1196,), Support: 0.02, Confidence: 0.88, Lift: 19.92
Rule: (1196, 1210) => (260,), Support: 0.02, Confidence: 0.89, Lift: 18.37
Rule: (4993, 5952) => (7153,), Support: 0.03, Confidence: 

,antecedent,consequent,support,confidence,lift
0,"(2115,)","(1291,)",0.023095,0.731092,19.813131
1,"(5952,)","(4993,)",0.034776,0.711957,12.650661
2,"(2571, 5952)","(7153,)",0.018051,0.764045,13.263398
3,"(2571, 7153)","(5952,)",0.018051,0.850000,17.401902
4,"(318, 32587)","(296,)",0.018848,0.771739,6.120297
5,"(2571, 7153)","(4993,)",0.018051,0.850000,15.103538
6,"(593, 858)","(296,)",0.018317,0.741935,5.883939
7,"(593, 858)","(318,)",0.019113,0.774194,7.860882
8,"(260, 1196)","(1210,)",0.022033,0.747748,19.292916
9,"(260, 1210)","(1196,)",0.022033,0.882979,19.917251


In [ ]:
print('By running Apriori  with support_threshold=0.018 and association rules with  min_confidence=0.8 we get')
rules = generate_association_rules(resultApriori, min_confidence=0.8)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


By running Apriori  with support_threshold=0.018 and association rules with  min_confidence=0.8 we get
The number of rules is 9
Rule: (2571, 7153) => (5952,), Support: 0.02, Confidence: 0.85, Lift: 17.40
Rule: (2571, 7153) => (4993,), Support: 0.02, Confidence: 0.85, Lift: 15.10
Rule: (260, 1210) => (1196,), Support: 0.02, Confidence: 0.88, Lift: 19.92
Rule: (1196, 1210) => (260,), Support: 0.02, Confidence: 0.89, Lift: 18.37
Rule: (4993, 5952) => (7153,), Support: 0.03, Confidence: 0.83, Lift: 14.44
Rule: (4993, 7153) => (5952,), Support: 0.03, Confidence: 0.89, Lift: 18.29
Rule: (5952, 7153) => (4993,), Support: 0.03, Confidence: 0.89, Lift: 15.75
Rule: (858, 4878) => (318,), Support: 0.02, Confidence: 0.80, Lift: 8.15
Rule: (2571, 5952) => (4993,), Support: 0.02, Confidence: 0.87, Lift: 15.37


,antecedent,consequent,support,confidence,lift
0,"(2571, 7153)","(5952,)",0.018051,0.850000,17.401902
1,"(2571, 7153)","(4993,)",0.018051,0.850000,15.103538
2,"(260, 1210)","(1196,)",0.022033,0.882979,19.917251
3,"(1196, 1210)","(260,)",0.022033,0.892473,18.371291
4,"(4993, 5952)","(7153,)",0.028935,0.832061,14.444120
5,"(4993, 7153)","(5952,)",0.028935,0.893443,18.291295
6,"(5952, 7153)","(4993,)",0.028935,0.886179,15.746395
7,"(858, 4878)","(318,)",0.018317,0.802326,8.146524
8,"(2571, 5952)","(4993,)",0.020441,0.865169,15.373066


In [ ]:
resultApriori = aprioriFromScratch(movie_baskets, support_threshold=0.008)
resultApriori






/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The number of frequents items is  23816


<ipython-input-11-a88d1ea5498f>:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, df], ignore_index=True)


,ItemSet,Support
0,{1},0.038758
1,{2},0.011680
2,{6},0.018317
3,{10},0.011415
4,{16},0.017521
...,...,...
23811,"{5952, 4993, 7153, 260, 1210, 2571, 1196}",0.010088
23812,"{2115, 2916, 485, 1240, 1291, 589, 1391}",0.008229
23813,"{1387, 2916, 485, 1240, 1291, 2028, 589}",0.008229
23814,"{1291, 2115, 2916, 485, 1240, 1387, 589}",0.008229


In [ ]:
print('By running Apriori  with support_threshold=0.008 and association rules with  min_confidence=0.8 we get')

rules = generate_association_rules(resultApriori, min_confidence=0.8)
rules

By running Apriori  with support_threshold=0.008 and association rules with  min_confidence=0.8 we get


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
Rule: (1200, 1214, 4993, 5952) => (1196,), Support: 0.01, Confidence: 0.82, Lift: 18.40
Rule: (485, 1291) => (589, 1391, 2916), Support: 0.01, Confidence: 0.84, Lift: 81.34
Rule: (485, 1391) => (589, 1291, 2916), Support: 0.01, Confidence: 0.89, Lift: 77.87
Rule: (485, 589, 1291) => (1391, 2916), Support: 0.01, Confidence: 0.86, Lift: 74.04
Rule: (485, 589, 1391) => (1291, 2916), Support: 0.01, Confidence: 0.97, Lift: 65.23
Rule: (485, 589, 2916) => (1291, 1391), Support: 0.01, Confidence: 0.80, Lift: 68.49
Rule: (485, 1291, 1391) => (589, 2916), Support: 0.01, Confidence: 0.97, Lift: 52.18
Rule: (485, 1291, 2916) => (589, 1391), Support: 0.01, Confidence: 0.84, Lift: 67.49
Rule: (485, 1391, 2916) => (589, 1291), Support: 0.01, Confidence: 0.94, Lift: 59.09
Rule: (589, 1291, 1391) => (485, 2916), Support: 0.01, Confidence: 0.89, Lift: 77.87
Rule: (589, 1391, 2916) => (485, 1291), Support: 0.01, Confidence: 0.82, Lift: 81.34
Rule: (1291

,antecedent,consequent,support,confidence,lift
0,"(107,)","(2115,)",0.010088,0.826087,26.150164
1,"(2383,)","(5785,)",0.008495,0.864865,66.488693
2,"(3256,)","(1089,)",0.010088,0.808511,11.897108
3,"(2383,)","(2959,)",0.008229,0.837838,7.514607
4,"(861,)","(1206,)",0.008495,0.914286,14.532128
...,...,...,...,...,...
20974,"(380, 485, 589, 1240, 1391, 2916)","(1291,)",0.008229,1.000000,27.100719
20975,"(380, 485, 589, 1291, 1391, 2916)","(1240,)",0.008229,1.000000,23.253086
20976,"(380, 485, 1240, 1291, 1391, 2916)","(589,)",0.008229,1.000000,19.518135
20977,"(380, 589, 1240, 1291, 1391, 2916)","(485,)",0.008229,1.000000,68.490909


In [ ]:
rules = generate_association_rules(resultApriori, min_confidence=0.8)
rules
print('By running Apriori  with support_threshold=0.008 and association rules with  min_confidence=0.9 we get')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
Rule: (485, 1291) => (589, 1391, 2916), Support: 0.01, Confidence: 0.84, Lift: 81.34
Rule: (485, 1391) => (589, 1291, 2916), Support: 0.01, Confidence: 0.89, Lift: 77.87
Rule: (485, 589, 1291) => (1391, 2916), Support: 0.01, Confidence: 0.86, Lift: 74.04
Rule: (485, 589, 1391) => (1291, 2916), Support: 0.01, Confidence: 0.97, Lift: 65.23
Rule: (485, 589, 2916) => (1291, 1391), Support: 0.01, Confidence: 0.80, Lift: 68.49
Rule: (485, 1291, 1391) => (589, 2916), Support: 0.01, Confidence: 0.97, Lift: 52.18
Rule: (485, 1291, 2916) => (589, 1391), Support: 0.01, Confidence: 0.84, Lift: 67.49
Rule: (485, 1391, 2916) => (589, 1291), Support: 0.01, Confidence: 0.94, Lift: 59.09
Rule: (589, 1291, 1391) => (485, 2916), Support: 0.01, Confidence: 0.89, Lift: 77.87
Rule: (589, 1391, 2916) => (485, 1291), Support: 0.01, Confidence: 0.82, Lift: 81.34
Rule: (1291, 1391, 2916) => (485, 589), Support: 0.01, Confidence: 0.91, Lift: 82.00
Rule: (485, 58

In [ ]:
print('By running Apriori  with support_threshold=0.008 and association rules with  min_confidence=0.9 we get')

rules = generate_association_rules(resultApriori, min_confidence=0.9)
rules

By running Apriori  with support_threshold=0.008 and association rules with  min_confidence=0.9 we get


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
Rule: (1240, 1387, 1391) => (2916,), Support: 0.01, Confidence: 0.97, Lift: 28.29
Rule: (1387, 1391, 2916) => (1240,), Support: 0.01, Confidence: 1.00, Lift: 23.25
Rule: (50, 356, 2571) => (318,), Support: 0.01, Confidence: 0.91, Lift: 9.23
Rule: (32, 1222, 2959) => (296,), Support: 0.01, Confidence: 0.91, Lift: 7.25
Rule: (32, 1617, 2420) => (50,), Support: 0.01, Confidence: 0.94, Lift: 12.02
Rule: (1214, 2571, 7153) => (5952,), Support: 0.01, Confidence: 0.92, Lift: 18.86
Rule: (589, 1291, 1387) => (2028,), Support: 0.01, Confidence: 0.91, Lift: 19.80
Rule: (1200, 2571, 7153) => (1214,), Support: 0.01, Confidence: 0.97, Lift: 19.26
Rule: (485, 2028) => (1291, 2916), Support: 0.01, Confidence: 0.97, Lift: 65.23
Rule: (485, 1291, 2028) => (2916,), Support: 0.01, Confidence: 1.00, Lift: 29.20
Rule: (485, 2028, 2916) => (1291,), Support: 0.01, Confidence: 1.00, Lift: 27.10
Rule: (380, 2115, 6539) => (1240,), Support: 0.01, Confidence: 0.

,antecedent,consequent,support,confidence,lift
0,"(861,)","(1206,)",0.008495,0.914286,14.532128
1,"(861,)","(2959,)",0.008760,0.942857,8.456531
2,"(861,)","(1089,)",0.008760,0.942857,13.873996
3,"(380, 2022)","(2115,)",0.008229,0.968750,30.666229
4,"(1387, 7318)","(2115,)",0.008495,0.914286,28.942137
...,...,...,...,...,...
7507,"(380, 485, 589, 1240, 1391, 2916)","(1291,)",0.008229,1.000000,27.100719
7508,"(380, 485, 589, 1291, 1391, 2916)","(1240,)",0.008229,1.000000,23.253086
7509,"(380, 485, 1240, 1291, 1391, 2916)","(589,)",0.008229,1.000000,19.518135
7510,"(380, 589, 1240, 1291, 1391, 2916)","(485,)",0.008229,1.000000,68.490909


In [ ]:
print('By running Apriori  with support_threshold=0.008 and association rules with  min_confidence=0.95 we get')

rules = generate_association_rules(resultApriori, min_confidence=0.95)
rules

By running Apriori  with support_threshold=0.008 and association rules with  min_confidence=0.95 we get


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The number of rules is 2964
Rule: (380, 2022) => (2115,), Support: 0.01, Confidence: 0.97, Lift: 30.67
Rule: (1089, 2383) => (5785,), Support: 0.01, Confidence: 0.97, Lift: 74.55
Rule: (2383, 5785) => (1089,), Support: 0.01, Confidence: 1.00, Lift: 14.71
Rule: (485, 2115) => (2916,), Support: 0.01, Confidence: 1.00, Lift: 29.20
Rule: (380, 7318) => (589,), Support: 0.01, Confidence: 1.00, Lift: 19.52
Rule: (107, 2022) => (2115,), Support: 0.01, Confidence: 0.97, Lift: 30.70
Rule: (107, 589) => (1291,), Support: 0.01, Confidence: 0.97, Lift: 26.28
Rule: (1206, 53121) => (1089,), Support: 0.01, Confidence: 0.97, Lift: 14.26
Rule: (1214, 5785) => (2959,), Support: 0.01, Confidence: 1.00, Lift: 8.97
Rule: (924, 5952) => (4993,), Support: 0.01, Confidence: 0.96, Lift: 17.01
Rule: (485, 589) => (1240,), Support: 0.01, Confidence: 0.95, Lift: 22.15
Rule: (485, 1240) => (589,), Support: 0.01, Confidence: 0.95, Lift: 18.59
Rule: (1617, 2115) => (1291,), Support: 0.01, Confidence: 0.95, Lift: 25

,antecedent,consequent,support,confidence,lift
0,"(380, 2022)","(2115,)",0.008229,0.968750,30.666229
1,"(1089, 2383)","(5785,)",0.008495,0.969697,74.547928
2,"(2383, 5785)","(1089,)",0.008495,1.000000,14.714844
3,"(485, 2115)","(2916,)",0.009822,1.000000,29.201550
4,"(380, 7318)","(589,)",0.008495,1.000000,19.518135
...,...,...,...,...,...
2959,"(380, 485, 589, 1240, 1391, 2916)","(1291,)",0.008229,1.000000,27.100719
2960,"(380, 485, 589, 1291, 1391, 2916)","(1240,)",0.008229,1.000000,23.253086
2961,"(380, 485, 1240, 1291, 1391, 2916)","(589,)",0.008229,1.000000,19.518135
2962,"(380, 589, 1240, 1291, 1391, 2916)","(485,)",0.008229,1.000000,68.490909


In [ ]:
print('By running Apriori  with support_threshold=0.008 and association rules with  min_confidence=0.98 we get')

rules = generate_association_rules(resultApriori, min_confidence=0.98)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


By running Apriori  with support_threshold=0.008 and association rules with  min_confidence=0.98 we get
The number of rules is 1140
Rule: (2383, 5785) => (1089,), Support: 0.01, Confidence: 1.00, Lift: 14.71
Rule: (485, 2115) => (2916,), Support: 0.01, Confidence: 1.00, Lift: 29.20
Rule: (380, 7318) => (589,), Support: 0.01, Confidence: 1.00, Lift: 19.52
Rule: (1214, 5785) => (2959,), Support: 0.01, Confidence: 1.00, Lift: 8.97
Rule: (2916, 7318) => (1291,), Support: 0.01, Confidence: 1.00, Lift: 27.10
Rule: (107, 380) => (1291,), Support: 0.01, Confidence: 1.00, Lift: 27.10
Rule: (380, 2022) => (589,), Support: 0.01, Confidence: 1.00, Lift: 19.52
Rule: (2115, 7318) => (1291,), Support: 0.01, Confidence: 1.00, Lift: 27.10
Rule: (107, 1240) => (1291,), Support: 0.01, Confidence: 1.00, Lift: 27.10
Rule: (107, 485) => (2916,), Support: 0.01, Confidence: 1.00, Lift: 29.20
Rule: (1095, 1206) => (1089,), Support: 0.01, Confidence: 1.00, Lift: 14.71
Rule: (485, 2115) => (589,), Support: 0.01,

,antecedent,consequent,support,confidence,lift
0,"(2383, 5785)","(1089,)",0.008495,1.0,14.714844
1,"(485, 2115)","(2916,)",0.009822,1.0,29.201550
2,"(380, 7318)","(589,)",0.008495,1.0,19.518135
3,"(1214, 5785)","(2959,)",0.008495,1.0,8.969048
4,"(2916, 7318)","(1291,)",0.008760,1.0,27.100719
...,...,...,...,...,...
1135,"(380, 485, 589, 1240, 1291, 1391)","(2916,)",0.008229,1.0,29.201550
1136,"(380, 485, 589, 1240, 1391, 2916)","(1291,)",0.008229,1.0,27.100719
1137,"(380, 485, 589, 1291, 1391, 2916)","(1240,)",0.008229,1.0,23.253086
1138,"(380, 485, 1240, 1291, 1391, 2916)","(589,)",0.008229,1.0,19.518135


It seems that the last run with $minimum support=0.008$ and $confidence=0.98$  enable us to find a significan amount of reliable rules for the reccoendation.

To compare the reccomendation via association rules and apriori to the clustering method we filter the rules having as antecedent the movieID of the movie Fight Club (2959) and figure out if the reccomendation based on the rules based on this movie lead to the the same movies found by employing the clustering methods.

In [ ]:
rules[rules['antecedent'].apply(lambda x: 2959 in x if isinstance(x, tuple) else False)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedent,consequent,support,confidence,lift
18,"(519, 2959)","(1089,)",0.008495,1.000000,14.714844
70,"(1206, 2353, 2959)","(1089,)",0.008495,1.000000,14.714844
149,"(2959, 3578, 7153)","(4993,)",0.008495,1.000000,17.768868
151,"(1206, 2959, 3440)","(1089,)",0.008229,1.000000,14.714844
163,"(2959, 5952, 7153)","(4993,)",0.014070,0.981481,17.439815
191,"(2959, 5952, 7361)","(4993,)",0.008229,1.000000,17.768868
195,"(2959, 6539, 7153)","(4993,)",0.008495,1.000000,17.768868
208,"(1089, 1610, 2959)","(1214,)",0.008495,1.000000,19.826316
225,"(1210, 2959, 7153)","(5952,)",0.008495,1.000000,20.472826
244,"(1210, 2959, 7153)","(4993,)",0.008495,1.000000,17.768868


From the result above, Fight Club alone doesn't have strong associations to recommend other movies, but when combined with movies that appear together frequently, the system can then confidently recommend movies with IDs 1089, 4993, 1214, and 5952. These movies are likely to be viewed by users who also watch Fight Club, based on past viewing behaviors.